# FAISS DB
- Facebook AI similarity Search

In [ ]:
# !pip install faiss-cpu
# !pip install faiss-gpu

In [ ]:
# !pip install numpy==1.24.0


### 영화 추천 시스템

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# movies = [
#   {
#     "id": "M001",
#     "title": "인셉션",
#     "description": "꿈과 현실의 경계를 넘나드는 특수 보안 전문가가 타인의 꿈에 침투하여 생각을 훔치는 임무를 수행하지만, 이번에는 생각을 심어야 하는 '인셉션' 작전에 도전한다. 현실과 꿈, 시간의 경계가 모호해지는 가운데 그의 과거가 미션의 성공을 위협한다."
#   },
#   {
#     "id": "M002",
#     "title": "기생충",
#     "description": "반지하 집에 살며 가족 모두 일용직으로 근근이 살아가는 기우네 가족. 우연한 기회로 기우는 부유한 박 사장네 딸의 과외 선생님이 되고, 차츰 온 가족이 박 사장네 집에 침투하게 된다. 두 가족의 만남 이후 예상치 못한 사건이 연이어 일어나며 걷잡을 수 없는 상황으로 번져간다."
#   },
#   {
#     "id": "M003",
#     "title": "어벤져스: 엔드게임",
#     "description": "타노스의 핑거 스냅으로 우주 생명체의 절반이 사라진 후, 남은 어벤져스 멤버들은 사라진 동료들을 되찾기 위해 마지막 희망을 걸고 타임 하이스트 작전을 펼친다. 과거로 시간 여행을 떠나 인피니티 스톤을 모으고 타노스에 맞서 최후의 전투를 벌이는 히어로들의 운명이 결정된다."
#   },
#   {
#     "id": "M004",
#     "title": "라라랜드",
#     "description": "꿈을 좇는 재즈 피아니스트 세바스찬과 배우 지망생 미아가 우연히 만나 사랑에 빠진다. 서로의 예술적 열정을 응원하며 꿈을 향해 나아가지만, 성공의 기로에 선 두 사람은 꿈과 사랑 사이에서 어려운 선택을 해야 한다. 황홀한 음악과 춤이 어우러진 현대적 뮤지컬 영화."
#   },
#   {
#     "id": "M005",
#     "title": "아바타",
#     "description": "지구의 자원 고갈로 판도라 행성의 자원을 채취하려는 인간들. 그 과정에서 현지 종족 나비족과 충돌하게 된다. 전직 해병대원 제이크 설리는 아바타 프로그램을 통해 나비족의 모습으로 침투하지만, 그들의 삶과 문화를 알게 되면서 자신의 임무와 나비족 사이에서 갈등하게 된다."
#   },
#   {
#     "id": "M006",
#     "title": "올드보이",
#     "description": "어느 날 갑자기 납치되어 15년간 한 방에 감금된 오대수. 그는 자신을 가둔 이유도 모른 채 복수를 다짐하며 탈출에 성공한다. 그를 도와주는 미도와 함께 진실을 찾아가는 과정에서 충격적인 비밀과 마주하게 된다. 복수와 용서, 금기의 주제를 다룬 복수 3부작의 대표작."
#   },
#   {
#     "id": "M007",
#     "title": "인터스텔라",
#     "description": "지구의 자원이 고갈되어 인류의 생존이 위협받는 미래, 전직 나사 파일럿 쿠퍼는 인류의 새로운 거주지를 찾기 위해 웜홀을 통과하는 우주 탐사에 참여한다. 미지의 행성들을 탐험하는 동안 상대성 이론에 따라 지구에서는 수십 년이 흘러, 그는 어린 딸과의 약속과 인류 구원 사이에서 고뇌한다."
#   },
#   {
#     "id": "M008",
#     "title": "매트릭스",
#     "description": "컴퓨터 프로그래머 네오는 자신이 살고 있는 세계가 인공지능에 의해 만들어진 가상현실 '매트릭스'라는 충격적인 진실을 깨닫는다. 인류의 구원자로 예언된 그는 모피어스와 트리니티의 도움을 받아 기계와의 전쟁에 뛰어들며, 자신의 능력을 깨닫고 인류를 해방시키기 위한 여정을 시작한다."
#   },
#   {
#     "id": "M009",
#     "title": "조커",
#     "description": "고담 시티에서 파티 클라운으로 일하며 희망 없는 삶을 살아가는 아서 플렉. 사회의 냉대와 끊임없는 모욕 속에서 정신적 질환을 앓던 그는 점차 자신의 어두운 내면에 굴복하며 악명 높은 범죄자 조커로 변모해간다. 사회의 부조리와 광기를 다룬 빌런의 기원 이야기."
#   },
#   {
#     "id": "M010",
#     "title": "겨울왕국",
#     "description": "태어날 때부터 얼음을 다루는 신비한 힘을 가진 엘사는 자신의 능력을 두려워한 나머지 왕국을 영원한 겨울로 만들고 도망친다. 여동생 안나는 언니를 찾아 험난한 여정을 떠나고, 빙산 일꾼 크리스토프와 눈사람 올라프의 도움으로 엘사에게 도달한다. 진정한 사랑의 의미를 깨닫게 되는 자매의 감동적인 이야기."
#   },
#   {
#     "id": "M011",
#     "title": "1917",
#     "description": "제1차 세계대전 당시, 영국군 두 병사가 1,600명의 동료 군인들의 목숨을 구하기 위해 적진을 횡단하는 위험한 임무를 맡는다. 단 하루 안에 중요한 메시지를 전달해야 하는 그들의 사투를 마치 한 번의 연속된 쇼트처럼 표현한 전쟁 영화로, 전장의 공포와 인간성을 생생하게 담아냈다."
#   },
#   {
#     "id": "M012",
#     "title": "미나리",
#     "description": "1980년대 미국 아칸소로 이주한 한국인 가족의 이야기. 아메리칸 드림을 좇는 아버지 제이콥은 자신만의 농장을 일구려 하고, 어머니 모니카는 낯선 환경에 적응하는 어려움을 겪는다. 할머니가 미국에 합류하면서 손자 데이비드와 특별한 유대감을 형성하는 가운데, 가족은 새로운 삶의 터전에서 희망과 좌절을 경험한다."
#   },
#   {
#     "id": "M013",
#     "title": "테넷",
#     "description": "시간의 흐름을 뒤집는 '역행' 기술을 둘러싼 음모에 맞서 싸우는 주인공의 이야기. 세계 3차 대전보다 더 끔찍한 무언가를 막기 위해 그는 미래에서 온 정보를 바탕으로 시간을 거스르는 여정을 시작한다. 크리스토퍼 놀란 감독의 복잡한 시간 구조와 화려한 액션이 특징인 SF 스릴러."
#   },
#   {
#     "id": "M014",
#     "title": "킹스맨: 시크릿 에이전트",
#     "description": "거리의 불량 청년 에그시는 비밀 독립 정보기관 '킹스맨'의 요원 해리 하트에게 발탁되어 엘리트 스파이가 되기 위한 훈련을 받는다. 그러던 중 세계적인 IT 기업가 발렌타인이 인류의 대부분을 제거하려는 음모를 꾸미고, 에그시는 세계를 구하기 위한 임무에 투입된다."
#   },
#   {
#     "id": "M015",
#     "title": "서울의 봄",
#     "description": "1979년 10.26 사건 이후 12월 12일에 벌어진 군사 쿠데타를 다룬 작품. 혼란스러운 정국 속에서 쿠데타 세력과 이에 맞서는 수도경비사령관 사이의 팽팽한 대립을 그린다. 대한민국 현대사의 중요한 전환점을 실화 기반으로 재구성한 정치 스릴러로, 민주주의의 위기와 권력 투쟁을 긴장감 있게 담아냈다."
#   },
#   {
#     "id": "M016",
#     "title": "그래비티",
#     "description": "우주 임무 중 폭발 사고로 인해 우주 공간에 고립된 두 우주 비행사의 생존을 위한 사투를 그린 작품. 경험 많은 우주 비행사 맷 코왈스키와 신참 라이언 스톤 박사는 지구로 귀환하기 위해 산소와 연료가 제한된 절체절명의 상황에서 극한의 선택을 해야 한다."
#   },
#   {
#     "id": "M017",
#     "title": "마션",
#     "description": "화성 탐사 도중 모래폭풍으로 인해 팀원들에게 사망한 것으로 착각되어 혼자 화성에 남겨진 우주 비행사 마크 와트니. 제한된 보급품으로 화성에서 살아남기 위해 그는 자신의 식물학 지식을 활용하고, 지구와의 통신을 재개하기 위해 고군분투한다. 인간의 생존 본능과 과학적 지식, 인류의 협력을 그린 SF 영화."
#   },
#   {
#     "id": "M018",
#     "title": "쇼생크 탈출",
#     "description": "1940년대, 아내와 그녀의 정부를 살해한 혐의로 종신형을 선고받은 은행가 앤디 듀프레인은 쇼생크 교도소에 수감된다. 그는 자신의 결백을 주장하지만 아무도 믿어주지 않는 가운데, 교도소 생활에 적응하며 동료 죄수 레드와 특별한 우정을 쌓는다. 긴 세월 동안 그는 자유를 향한 희망을 잃지 않고 탈출을 계획한다."
#   },
#   {
#     "id": "M019",
#     "title": "토이 스토리",
#     "description": "장난감들이 인간이 없을 때 살아 움직인다는 상상력을 바탕으로 한 픽사의 첫 장편 애니메이션. 카우보이 인형 우디는 앤디의 방에서 가장 사랑받는 장난감이었지만, 새로운 우주비행사 장난감 버즈 라이트이어의 등장으로 위기를 맞는다. 경쟁자였던 두 장난감은 위험에 처하면서 진정한 우정을 발견하게 된다."
#   },
#   {
#     "id": "M020",
#     "title": "다크 나이트",
#     "description": "배트맨이 경찰 고든 경감, 신임 지방 검사 하비 덴트와 함께 고담시의 범죄 조직을 소탕하려 하자, 알 수 없는 과거를 지닌 조커가 등장해 도시를 혼란에 빠뜨린다. 무자비한 범죄와 광기, 혼돈을 상징하는 조커에 맞서 배트맨은 영웅으로서의 자신의 정체성과 한계에 대해 고민하게 된다. 크리스토퍼 놀란 감독의 배트맨 3부작 중 두 번째 작품."
#   }
# ]

In [ ]:
# descriptions = [movie['description'] for movie in movies]
# desc_embed = np.array([model.encode(description) for description in descriptions],
#          dtype='float32')

In [ ]:
# dim = desc_embed.shape[1]
# idx = faiss.IndexFlatL2(dim)
# idx.add(desc_embed)

In [ ]:
# query_text = '탈출하는 이야기'
# query_embed = np.array([model.encode(query_text)], dtype='float32')

In [ ]:
# top_n = 5
# distances, indices = idx.search(query_embed, 5)

In [ ]:
# print(distances)
# print(indices)

In [ ]:
# for i in range(top_n):
#     movie_id = indices[0][i]
#     print(f"{i+1}번쨰 추천: {movies[movie_id]['title']} (유사도 거리: {distances[0][i]:.2f})")

In [ ]:
#!pip install beautifulsoup4

### 사용자 맞춤 뉴스 추천

In [ ]:
import requests
from bs4 import BeautifulSoup

# 스크랩한 뉴스 정보를 담을 NewsEntry class
class NewsEntry:
    def __init__(self, title, content, href):
        self.title = title
        self.content = content
        self.href = href

    def __repr__(self):
        return f'NewsEntry<title={self.title}, content={self.content[:30]}..., href={self.href}>'

# 1. request -> url 요청
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=it"

response = requests.get(url)

# 2. html 응답
html = response.text

# 3. BeautifulSoup 객체 생성
bs = BeautifulSoup(html, 'html.parser')

# 4. div.news_contents 반복순회
news_contents = bs.select('div.news_contents')
print(f"{len(news_contents)}개의 뉴스를 찾았습니다.")

# 5. 뉴스 제목, 링크, 내용 추출
news_list = []
for i, news_content in enumerate(news_contents[:10]):  # 상위 5개만 처리
    # 제목과 링크 추출
    a_tag = news_content.select_one("a.news_tit")
    title = a_tag.text
    href = a_tag['href']
    
    print(f"[{i+1}/5] '{title}' 내용 가져오는 중...")
    
    # 뉴스 본문 가져오기
    article_response = requests.get(href)
    article_html = article_response.text
    article_bs = BeautifulSoup(article_html, 'html.parser')
    
    # 네이버 뉴스인 경우
    content = "내용을 가져올 수 없습니다."
    if 'news.naver.com' in href:
        content_element = article_bs.select_one('#dic_area') or article_bs.select_one('#articleBodyContents')
        if content_element:
            content = content_element.get_text().strip()
    
    # 다른 뉴스 사이트인 경우 - 일반적인 본문 영역 시도
    else:
        content_candidates = [
            article_bs.select_one('article'),
            article_bs.select_one('.article_body'),
            article_bs.select_one('.article-body'),
            article_bs.select_one('.news-content'),
            article_bs.select_one('#articleBody')
        ]
        
        for candidate in content_candidates:
            if candidate:
                content = candidate.get_text().strip()
                break
    
    # 뉴스 엔트리 생성 및 추가
    news_entry = NewsEntry(title, content, href)
    news_list.append(news_entry)

# 6. 수집된 뉴스 출력
# for i, news in enumerate(news_list):
#     print(f"\n=== 뉴스 {i+1} ===")
#     print(f"제목: {news.title}")
#     print(f"내용 미리보기: {news.content[:150]}...")
#     print(f"원본 링크: {news.href}")


In [ ]:
descriptions = [news.content for news in news_list]
desc_embed = np.array(
    [model.encode(description) for description in descriptions],
    dtype='float32'
)

In [ ]:
dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(desc_embed)

In [ ]:
query_text = input('검색: ')
query_embed = np.array([model.encode(query_text)], dtype='float32')

In [ ]:
top_n = 3
distances, indices = idx.search(query_embed, 3)

In [ ]:
for i in range(top_n):
    movie_id = indices[0][i]
    print(f"{i+1}번째 추천: {movies[movie_id]['title']} (유사도 거리: {distances[0][i]:.2f})")